<a href="https://colab.research.google.com/github/AnulipiJana/Citation-Analysis--Research-Paper/blob/main/Citation_Network_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODULE3
- marge derived data
- preprocess merge data
- model training using
  1. XGBOOST
  2. Neural Network
- Test model

## Marge derived data
columns: id, betweenness, closeness, degree, eigenvector, abstract, authors, n_citation, references, title, venue, year

###Import required libraries

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
import matplotlib.pyplot as plt
import ast

###Upload required files and merge

In [2]:
from google.colab import files

uploaded = files.upload()  # Upload your dataset
df = pd.read_csv(list(uploaded.keys())[0])  # Read dataset into a pandas DataFrame

Saving filtered_1980.csv to filtered_1980.csv


In [3]:
df['references'] = df['references'].fillna('').astype(str)

In [4]:
from google.colab import files

uploaded = files.upload()  # Upload your dataset
coauthor_df = pd.read_csv(list(uploaded.keys())[0])  # Read dataset into a pandas DataFrame

Saving coauthor_centrality.csv to coauthor_centrality.csv


In [5]:
from google.colab import files

uploaded = files.upload()  # Upload your dataset
paper_citation_df = pd.read_csv(list(uploaded.keys())[0])  # Read dataset into a pandas DataFrame

Saving Paper_centrality.csv to Paper_centrality.csv


In [6]:
uploaded = files.upload()  # Upload your dataset
author_citation_df = pd.read_csv(list(uploaded.keys())[0])  # Read dataset into a pandas DataFrame

Saving author_centrality.csv to author_centrality.csv


In [9]:
uploaded = files.upload()  # Upload your dataset
hindex_df = pd.read_csv(list(uploaded.keys())[0])  # Read dataset into a pandas DataFrame

Saving h_index_results.csv to h_index_results.csv


In [7]:
# Merge paper-related data
paper_dataset = paper_citation_df.merge(df, on="id", how="left")
paper_dataset.to_csv("paper_dataset.csv", index=False)

In [10]:
# Merge author-related data
author_dataset = author_citation_df.merge(hindex_df, on="Author", how="left")
author_dataset = author_dataset.merge(coauthor_df, on="Author", how="left")
author_dataset.to_csv("author_dataset.csv", index=False)

In [11]:
from google.colab import files

# Download paper dataset
files.download("paper_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("author_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###Download the combined paper-related dataset

In [12]:
from google.colab import files
import pandas as pd

uploaded = files.upload()  # Upload your dataset
df = pd.read_csv(list(uploaded.keys())[0])  # Read dataset into a pandas DataFrame

Saving paper_dataset.csv to paper_dataset (1).csv


In [13]:
df = pd.read_csv('paper_dataset.csv')
df

,id,betweenness,closeness,degree,eigenvector,abstract,authors,n_citation,references,title,venue,year
0,4fca075d-965a-4022-8891-e9d1e326e464,0.0,0.000000,0.000146,0.000025,NaN,['Frederic B. Fitch'],50.0,[],Modal functions in two-valued logic,Journal of Symbolic Logic,1937.0
1,[],0.0,0.171850,0.171850,0.801998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b6b5baab-032f-4598-aeb2-f2bb7f12b4c7,0.0,0.000000,0.000146,0.000025,NaN,['W. V. Quine'],18.0,[],Logic based on inclusion and abstraction,Journal of Symbolic Logic,1937.0
3,f821fbaa-1f89-427f-b5e8-4c904a02e2e7,0.0,0.000000,0.000146,0.000025,NaN,['W. V. Quine'],21.0,['7dc8b493-dd60-4580-bcd2-cea02ca46473'],On the Theory of Types,Journal of Symbolic Logic,1938.0
4,['7dc8b493-dd60-4580-bcd2-cea02ca46473'],0.0,0.000292,0.000292,0.001388,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6845,49829369-a417-4a86-bc78-caca4b6832fc,0.0,0.000000,0.000146,0.000025,NaN,['Arto Salomaa'],50.0,"['b001b8d1-2cc3-4b88-9f82-0f0b0a871c8e', 'c0b1...",Grammatical Families,"international colloquium on automata, language...",1980.0
6846,"['b001b8d1-2cc3-4b88-9f82-0f0b0a871c8e', 'c0b1...",0.0,0.000146,0.000146,0.000707,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6847,7d9f36a2-738f-4152-b60b-9a135d76bbc1,0.0,0.000000,0.000146,0.000025,Parallel counters are multiple input circuits ...,['K. W. Current'],4.0,[],Pipelined Binary Parallel Counters Employing L...,IEEE Transactions on Computers,1980.0
6848,7b7b8ac1-d61f-4b87-b745-73352713e600,0.0,0.000000,0.000146,0.000025,"A new inference control, called random sample ...",['Dorothy E. Denning'],233.0,"['26394507-f6ed-45c4-93b8-470108a955d1', '2cb7...",Secure statistical databases with random sampl...,ACM Transactions on Database Systems,1980.0


## Preprocess merge data

In [14]:
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from collections import Counter
from tabulate import tabulate

In [15]:
# Convert 'references' and 'authors' columns from string to list
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val) if isinstance(val, str) else []
    except (SyntaxError, ValueError):
        return []

df["references"] = df["references"].apply(safe_literal_eval)
df["authors"] = df["authors"].apply(safe_literal_eval)

# Fix NULL values in Authors columns
df["authors"] = df["authors"].apply(lambda x: ', '.join(x) if isinstance(x, list) and x else "Unknown")
df["authors"] = df["authors"].fillna("Unknown")  # Handle NaN values

# Normalize Citation Count
df["normalized_citations"] = MinMaxScaler().fit_transform(df[['n_citation']])

# TF-IDF Vectorization for Paper Abstracts
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
abstract_vectors = vectorizer.fit_transform(df['abstract'].fillna(""))

# Ensure Precomputed Centralities Are Filled
centrality_features = ["betweenness", "closeness", "degree", "eigenvector"]
for feature in centrality_features:
    df[feature] = df[feature].fillna(0)

# Compute PageRank
if "pagerank" not in df.columns:
    G_paper = nx.DiGraph()
    for _, row in df.iterrows():
        paper_id = row["title"]
        for ref in row["references"]:
            G_paper.add_edge(paper_id, ref)
    pagerank_scores = nx.pagerank(G_paper)
    df["pagerank"] = df["title"].map(pagerank_scores).fillna(0)

# Compute H-Index for Authors
if "h_index" not in df.columns:
    def compute_hindex(author_papers):
        citations = sorted(author_papers, reverse=True)
        h_index = sum(c >= i+1 for i, c in enumerate(citations))
        return h_index
#dictionary author_hindex where each author maps to a list of their citation counts.
    author_hindex = {}
    for _, row in df.iterrows():
        for author in row["authors"].split(', '):  # Since authors are now strings
            if author not in author_hindex:
                author_hindex[author] = []
            author_hindex[author].append(row["n_citation"])
#For each paper, calculate the average h-index of its authors
    df["h_index"] = df["authors"].apply(lambda authors: np.mean([compute_hindex(author_hindex[a]) for a in authors.split(', ') if a in author_hindex]) if authors != "Unknown" else 0)

# Remove NaN or infinite values in `n_citation`
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["n_citation"])

# Ensure No NaN in Feature Columns
features = ["normalized_citations", "pagerank", "h_index", "betweenness", "closeness", "degree", "eigenvector"]
df[features] = df[features].fillna(0)

## Model Training

###  XGBoost

In [16]:
#Check how abstract_vectors was generated:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
abstract_vectors = vectorizer.fit_transform(df["abstract"].fillna(""))

In [17]:
#Confirm vector length matches df length:
print("Abstract vectors shape:", abstract_vectors.shape)
print("Current df length:", len(df))

Abstract vectors shape: (4491, 5000)
Current df length: 4491


In [18]:
# Prepare Data for Training
X = df[features]
y = df["n_citation"].astype(float)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Train XGBoost Model
model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.5, max_depth=500)
model.fit(X_train, y_train)

# Predict & Evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(" Model RMSE:", rmse)

 Model RMSE: 137.4427378514665


In [19]:
from tabulate import tabulate

# Recommendation Function
def recommend_papers(query, top_n=10):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, abstract_vectors).flatten()

    if len(similarities) != len(df):
        print(f"Warning: Similarity scores length ({len(similarities)}) does not match dataframe ({len(df)})")
        return None

    df['similarity'] = similarities
    df['score'] = df['similarity'] + df['pagerank'] + df['normalized_citations']

    # Drop or ignore authors
    top_papers = df.nlargest(top_n, 'score')[["title", "venue", "year", "score"]]

    print("\nTop Recommended Research Papers\n")
    print(tabulate(top_papers, headers="keys", tablefmt="fancy_grid", showindex=False))

    return top_papers

In [20]:
# **User Input for Paper Recommendations**
user_input = input("\nEnter research topic: ")
recommended_papers = recommend_papers(user_input, top_n=10)


Enter research topic: Cryptography

Top Recommended Research Papers

╒═════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════╤════════╤══════════╕
│ title                                                                               │ venue                                   │   year │    score │
╞═════════════════════════════════════════════════════════════════════════════════════╪═════════════════════════════════════════╪════════╪══════════╡
│ New Directions in Cryptography                                                      │ IEEE Transactions on Information Theory │   1976 │ 1.20511  │
├─────────────────────────────────────────────────────────────────────────────────────┼─────────────────────────────────────────┼────────┼──────────┤
│ The Concept of a Linguistic Variable and its Application to Approximate Reasoning-I │ Information Sciences                    │   1975 │ 0.722857 │
├─────────────────────────────

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from collections import Counter
import numpy as np
import re
from tabulate import tabulate

def clean_title(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    return text

# Clean titles
df["clean_title"] = df["title"].fillna("").apply(clean_title)

# Fit TF-IDF to extract high-weighted keywords (technical terms)
tfidf = TfidfVectorizer(stop_words='english', max_features=500)
tfidf_matrix = tfidf.fit_transform(df["clean_title"])
terms = tfidf.get_feature_names_out()

# Reverse map: term -> importance across all titles
term_scores = np.asarray(tfidf_matrix.sum(axis=0)).flatten()
term_importance = dict(zip(terms, term_scores))

# Function to extract highest scoring term from each title
def extract_top_tfidf_term(title):
    words = title.split()
    scored_terms = [(word, term_importance.get(word, 0)) for word in words if word not in ENGLISH_STOP_WORDS]
    if not scored_terms:
        return "unknown"
    return max(scored_terms, key=lambda x: x[1])[0]

# Assign most relevant domain based on TF-IDF
df["domain"] = df["clean_title"].apply(extract_top_tfidf_term)

# Predict future trends (print only tabulated part)
def predict_popular_domains():
    domain_counts = Counter(df["domain"])
    top_domains = domain_counts.most_common(10)

    print("\nPredicted Popular Research Domains\n")
    print(tabulate(
        [(domain, int(count * np.random.uniform(1.1, 1.5))) for domain, count in top_domains],
        headers=['Domain', 'Future Citations'],
        tablefmt='fancy_grid'
    ))

In [22]:
# Predict Most Popular Research Domains
predict_popular_domains()


Predicted Popular Research Domains

╒═════════════╤════════════════════╕
│ Domain      │   Future Citations │
╞═════════════╪════════════════════╡
│ systems     │                338 │
├─────────────┼────────────────────┤
│ computer    │                226 │
├─────────────┼────────────────────┤
│ design      │                189 │
├─────────────┼────────────────────┤
│ algorithm   │                188 │
├─────────────┼────────────────────┤
│ analysis    │                152 │
├─────────────┼────────────────────┤
│ theory      │                160 │
├─────────────┼────────────────────┤
│ data        │                147 │
├─────────────┼────────────────────┤
│ problem     │                126 │
├─────────────┼────────────────────┤
│ programming │                106 │
├─────────────┼────────────────────┤
│ logic       │                130 │
╘═════════════╧════════════════════╛


### Neural Network

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [24]:
# Convert 'references' and 'authors' columns from string to list
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val) if isinstance(val, str) else []
    except (SyntaxError, ValueError):
        return []

df["references"] = df["references"].apply(safe_literal_eval)
df["authors"] = df["authors"].apply(safe_literal_eval)

# Normalize citation count
df["normalized_citations"] = MinMaxScaler().fit_transform(df[['n_citation']])

# TF-IDF Vectorization for Paper Abstracts
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
abstract_vectors = vectorizer.fit_transform(df['abstract'].fillna(""))

# Load precomputed centrality values from CSV
centrality_features = ["betweenness", "closeness", "degree", "eigenvector"]
for feature in centrality_features:
    df[feature] = df[feature].fillna(0)  # Ensure no missing values

# Compute PageRank (If not already in the dataset)
if "pagerank" not in df.columns:
    G_paper = nx.DiGraph()
    for _, row in df.iterrows():
        paper_id = row["title"]
        for ref in row["references"]:
            G_paper.add_edge(paper_id, ref)

    pagerank_scores = nx.pagerank(G_paper)
    df["pagerank"] = df["title"].map(pagerank_scores).fillna(0)

# H-Index Calculation (If not precomputed)
if "h_index" not in df.columns:
    def compute_hindex(author_papers):
        citations = sorted(author_papers, reverse=True)
        h_index = sum(c >= i+1 for i, c in enumerate(citations))
        return h_index

    author_hindex = {}
    for _, row in df.iterrows():
        for author in row["authors"]:
            if author not in author_hindex:
                author_hindex[author] = []
            author_hindex[author].append(row["n_citation"])

    df["h_index"] = df["authors"].apply(lambda authors: np.mean([compute_hindex(author_hindex[a]) for a in authors]) if authors else 0)

# Remove NaN or infinite values in the target variable (n_citation)
df = df.replace([np.inf, -np.inf], np.nan)  # Convert infinities to NaN
df = df.dropna(subset=["n_citation"])

# Check for NaN in feature columns and fill with 0
features = ["normalized_citations", "pagerank", "h_index", "betweenness", "closeness", "degree", "eigenvector"]
df[features] = df[features].fillna(0)

In [25]:
# Prepare Data for Training
X = df[features]
y = df["n_citation"].astype(float)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Build Neural Network Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(32, activation='relu'),

    Dense(1)  # Regression Output
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
# Compile the Model
model.compile(optimizer='adam', loss='mse')

# Train the Model
model.fit(X_train, y_train, epochs=250, batch_size=32, validation_split=0.3, verbose=1)

Epoch 1/250
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 201803.9688 - val_loss: 132507.0938
Epoch 2/250
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 478080.9688 - val_loss: 131566.0625
Epoch 3/250
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 372776.0625 - val_loss: 130328.6406
Epoch 4/250
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 421372.5312 - val_loss: 129272.6953
Epoch 5/250
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 142919.3594 - val_loss: 130496.0469
Epoch 6/250
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 168222.4688 - val_loss: 128135.1016
Epoch 7/250
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 219279.8281 - val_loss: 117928.1094
Epoch 8/250
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 166474.2656 - val_loss: 98763.6094
Epoch 9/250
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 218446.5781 - val_loss: 84333.8125
Epoch 10/250
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 226209.2031 - val_loss: 59160.4414
Epoch 11/250
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/ste

In [27]:
# Predict & Evaluate
y_pred = model.predict(X_test).flatten()
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Neural Network Model RMSE:", rmse)

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Neural Network Model RMSE: 99.5471842306666


In [28]:
from tabulate import tabulate

# Recommendation Function
def recommend_papers(query, top_n=10):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, abstract_vectors).flatten()

    if len(similarities) != len(df):
        print(f"Warning: Similarity scores length ({len(similarities)}) does not match dataframe ({len(df)})")
        return None

    df['similarity'] = similarities
    df['score'] = df['similarity'] + df['pagerank'] + df['normalized_citations']

    # Drop or ignore authors
    top_papers = df.nlargest(top_n, 'score')[["title", "venue", "year", "score"]]

    print("\nTop Recommended Research Papers\n")
    print(tabulate(top_papers, headers="keys", tablefmt="fancy_grid", showindex=False))

    return top_papers

In [29]:
#  **User Input**
user_input = input("\nEnter research topic: ")
recommended_papers = recommend_papers(user_input, top_n=10)


Enter research topic: cryptography

Top Recommended Research Papers

╒═════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════╤════════╤══════════╕
│ title                                                                               │ venue                                   │   year │    score │
╞═════════════════════════════════════════════════════════════════════════════════════╪═════════════════════════════════════════╪════════╪══════════╡
│ New Directions in Cryptography                                                      │ IEEE Transactions on Information Theory │   1976 │ 1.20511  │
├─────────────────────────────────────────────────────────────────────────────────────┼─────────────────────────────────────────┼────────┼──────────┤
│ The Concept of a Linguistic Variable and its Application to Approximate Reasoning-I │ Information Sciences                    │   1975 │ 0.722857 │
├─────────────────────────────

# MODULE4
- upload dataset
- install libraries
- create user interface


## Upload Dataset

In [ ]:
from google.colab import files
import pandas as pd

uploaded = files.upload()  # Upload your dataset
df = pd.read_csv(list(uploaded.keys())[0])  # Read dataset into a pandas DataFrame

Saving paper_dataset.csv to paper_dataset (1).csv


In [ ]:
df = pd.read_csv('paper_dataset.csv')
df

## Install and import libraries

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
import matplotlib.pyplot as plt
import ast

In [ ]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.9 MB/s eta 0:00:00


## User Interface

In [ ]:
# UI-Based Paper Recommender System for Google Colab
# Must run in Google Colab

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import gradio as gr

# Safe literal eval for list columns
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val) if isinstance(val, str) else []
    except (SyntaxError, ValueError):
        return []

# Preprocessing Function
def preprocess_data(df):
    df["references"] = df["references"].apply(safe_literal_eval)
    df["authors"] = df["authors"].apply(safe_literal_eval)

    df["normalized_citations"] = MinMaxScaler().fit_transform(df[['n_citation']])

    centrality_features = ["betweenness", "closeness", "degree", "eigenvector"]
    for feature in centrality_features:
        df[feature] = df[feature].fillna(0)

    if "pagerank" not in df.columns:
        G_paper = nx.DiGraph()
        for _, row in df.iterrows():
            paper_id = row["title"]
            for ref in row["references"]:
                G_paper.add_edge(paper_id, ref)
        pagerank_scores = nx.pagerank(G_paper)
        df["pagerank"] = df["title"].map(pagerank_scores).fillna(0)

    if "h_index" not in df.columns:
        def compute_hindex(author_papers):
            citations = sorted(author_papers, reverse=True)
            h_index = sum(c >= i+1 for i, c in enumerate(citations))
            return h_index

        author_hindex = {}
        for _, row in df.iterrows():
            for author in row["authors"]:
                if author not in author_hindex:
                    author_hindex[author] = []
                author_hindex[author].append(row["n_citation"])

        df["h_index"] = df["authors"].apply(lambda authors: np.mean([compute_hindex(author_hindex[a]) for a in authors]) if authors else 0)

    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=["n_citation"])

    features = ["normalized_citations", "pagerank", "h_index", "betweenness", "closeness", "degree", "eigenvector"]
    df[features] = df[features].fillna(0)
    return df, features

# Train Neural Network
def train_model(df, features):
    X = df[features]
    y = df["n_citation"].astype(float)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        BatchNormalization(), Dropout(0.3),
        Dense(64, activation='relu'),
        BatchNormalization(), Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
    return model

# Main Recommendation Function
def build_recommender(df, model, vectorizer, abstract_vectors):
    def recommend(query):
        query_vec = vectorizer.transform([query])
        sim_scores = cosine_similarity(query_vec, abstract_vectors).flatten()
        df["similarity"] = sim_scores
        df["score"] = df["similarity"] + df["pagerank"] + df["normalized_citations"]
        df["authors"] = df["authors"].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))
        df["authors"] = df["authors"].fillna("Unknown")
        top_papers = df.nlargest(5, "score")[["title", "authors", "venue", "year", "score"]]
        return top_papers
    return recommend

# Load CSV in Colab and Initialize
# Run in a Colab cell before UI
df = pd.read_csv("paper_dataset.csv")
df, features = preprocess_data(df)
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
abstract_vectors = vectorizer.fit_transform(df["abstract"].fillna(""))
model = train_model(df, features)
recommender = build_recommender(df, model, vectorizer, abstract_vectors)

# Build Gradio UI
with gr.Blocks(theme=gr.themes.Soft(), css="body {background: #fefefe; font-family: 'Segoe UI';}") as demo:
    gr.Markdown("""
    # AI-Powered Research Paper Recommender
    Enter your research interest and get top 5 most relevant papers
    """)
    query = gr.Textbox(label="Enter Research Topic", placeholder="e.g., Neural networks for text generation")
    output = gr.Dataframe(headers=["Title", "Authors", "Venue", "Year", "Score"], wrap=True)
    btn = gr.Button("Recommend Papers")
    btn.click(fn=recommender, inputs=query, outputs=output)

# Launch in Colab
demo.launch(debug=False, share=True)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4957d031fbb3a3bdcd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
